In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import getpass
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
output_filename = "/content/dwarka_roads_dataset_large.csv"
dwarka_df = pd.read_csv(output_filename)

print(f"Data loaded from '{output_filename}' into dwarka_df")

Data loaded from '/content/dwarka_roads_dataset_large.csv' into dwarka_df


### Implement a two-stage supervised learning pipeline to predict `pothole_count` and `observed_travel_time_s` from a given dataset. The pipeline should predict rates (`potholes_per_km` and `time_per_km_s`) in the first stage using a count model (Gradient Boosting with Poisson) for potholes and a regression model for travel time, incorporating out-of-fold pothole predictions in the second stage to prevent data leakage. The final evaluation should be performed on the original targets (`pothole_count` and `observed_travel_time_s`) using metrics like MAE, RMSE, and calibration, potentially employing spatial K-fold cross-validation and hyperparameter tuning with Optuna or RandomizedSearchCV.

## Data preprocessing

###Calculate rates (potholes per km and travel time per km)

In [ ]:
print(dwarka_df.columns)

Index(['segment_id', 'road_name', 'geometry', 'start_lat', 'start_lon',
       'end_lat', 'end_lon', 'length_m', 'lane_count', 'lane_width_m',
       'speed_limit_kph', 'surface_type', 'pothole_count', 'pothole_sizes_cm',
       'pothole_roughness_score', 'road_condition_score',
       'observed_travel_time_s', 'estimated_travel_time_s',
       'avg_speed_kph_observed', 'vehicle_composition',
       'avg_traffic_density_veh_per_km', 'lighting', 'sidewalk_present',
       'parking_on_road', 'drainage_issue', 'signal_present_start',
       'signal_present_end', 'pedestrian_crossings', 'accident_count_1yr',
       'bus_stop_count', 'bus_routes', 'nearest_metro_station',
       'metro_distance_m', 'street_vendor_density',
       'avg_wait_time_at_signal_s', 'historical_variation',
       'legal_restrictions'],
      dtype='object')


In [ ]:
dwarka_df['potholes_per_km'] = dwarka_df['pothole_count'] / dwarka_df['length_m'] * 1000
dwarka_df['potholes_per_km'] = dwarka_df['potholes_per_km'].replace([np.inf, -np.inf], 0)

dwarka_df['time_per_km_s'] = dwarka_df['observed_travel_time_s'] / dwarka_df['length_m'] * 1000
dwarka_df['time_per_km_s'] = dwarka_df['time_per_km_s'].replace([np.inf, -np.inf], 0)


categorical_cols = dwarka_df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('geometry')
categorical_cols.remove('segment_id')

# Define features (X) and new target variables (y1 and y2)
features = dwarka_df.drop(['pothole_count', 'observed_travel_time_s', 'geometry', 'segment_id', 'potholes_per_km', 'time_per_km_s'], axis=1)
y1 = dwarka_df['potholes_per_km']
y2 = dwarka_df['time_per_km_s']


preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough' # Keep other columns (numerical)
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the features
X_processed = pipeline.fit_transform(features)


X_processed_dense = X_processed.toarray()
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
X_processed_df = pd.DataFrame(X_processed_dense, columns=feature_names)

# Split the data into training and testing sets for the first stage (predicting rates)
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(
    X_processed_df, y1, y2, test_size=0.2, random_state=42
)

print("Data splitting and rate calculation complete for the first stage.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y1_train shape: {y1_train.shape}")
print(f"y1_test shape: {y1_test.shape}")
print(f"y2_train shape: {y2_train.shape}")
print(f"y2_test shape: {y2_test.shape}")

Data splitting and rate calculation complete for the first stage.
X_train shape: (160, 627)
X_test shape: (40, 627)
y1_train shape: (160,)
y1_test shape: (40,)
y2_train shape: (160,)
y2_test shape: (40,)


## Stage 1 model training (count model)

### Train a gradient boosting model with Poisson `potholes_per_km`.


In [ ]:
X_train.columns = ["".join (c if c.isalnum() or c in ['_'] else "_" for c in str(x)) for x in X_train.columns]
X_test.columns = ["".join (c if c.isalnum() or c in ['_'] else "_" for c in str(x)) for x in X_test.columns]

# Instantiate the chosen regression model with Poisson objective
stage1_count_model = xgb.XGBRegressor(objective='count:poisson', random_state=42)
stage1_count_model.fit(X_train, y1_train)
print("Stage 1 count model (XGBoost Regressor with Poisson objective) trained.")

Stage 1 count model (XGBoost Regressor with Poisson objective) trained.


## Stage 1 model evaluation


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y1_pred_stage1 = stage1_count_model.predict(X_test)
mse_potholes_per_km = mean_squared_error(y1_test, y1_pred_stage1)
r2_potholes_per_km = r2_score(y1_test, y1_pred_stage1)

print(f"Mean Squared Error (MSE) for potholes_per_km prediction: {mse_potholes_per_km}")
print(f"R-squared (R2) for potholes_per_km prediction: {r2_potholes_per_km}")

Mean Squared Error (MSE) for potholes_per_km prediction: 5.8095527013275285
R-squared (R2) for potholes_per_km prediction: 0.8061777749685815


## Stage 2 data preparation

### Generate out-of-fold predictions for `potholes_per_km` from the Stage 1 model using KFold cross-validation & use these to create the training data for Stage 2.

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof_predictions = np.zeros(X_train.shape[0])

for train_index, val_index in kf.split(X_train):
    X_inner_train, X_inner_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y1_inner_train = y1_train.iloc[train_index]

    stage1_inner_model = xgb.XGBRegressor(objective='count:poisson', random_state=42)
    stage1_inner_model.fit(X_inner_train, y1_inner_train)

    val_predictions = stage1_inner_model.predict(X_inner_val)
    oof_predictions[val_index] = val_predictions

X_train_stage2 = X_train.copy()
X_train_stage2['predicted_potholes_per_km'] = oof_predictions

y1_test_pred_stage1 = stage1_count_model.predict(X_test)

X_test_stage2 = X_test.copy()
X_test_stage2['predicted_potholes_per_km'] = y1_test_pred_stage1

print("Data prepared for Stage 2 with out-of-fold predictions for training.")
print(f"X_train_stage2 shape: {X_train_stage2.shape}")
print(f"X_test_stage2 shape: {X_test_stage2.shape}")

Data prepared for Stage 2 with out-of-fold predictions for training.
X_train_stage2 shape: (160, 628)
X_test_stage2 shape: (40, 628)


## Stage 2 model training

### Train a XGBoost regression model to predict `time_per_km_s` using the out-of-fold  `potholes_per_km`.


In [ ]:
stage2_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
stage2_model.fit(X_train_stage2, y2_train)

print("Stage 2 model (XGBoost Regressor) trained successfully.")

Stage 2 model (XGBoost Regressor) trained successfully.


## Hyperparameter tuning

### Use Optuna or RandomizedSearchCV to tune the hyperparameters of the models.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Define parameter distributions for Stage 1 model (XGBoost Regressor with Poisson objective)
param_dist_stage1 = {
    'n_estimators': randint(100, 500),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'lambda': uniform(1, 2),
    'alpha': uniform(0, 1)
}

random_search_stage1 = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(objective='count:poisson', random_state=42),
    param_distributions=param_dist_stage1, n_iter=50,
    scoring='neg_mean_squared_error', cv=5, verbose=1, random_state=42,
    n_jobs=-1
)

random_search_stage1.fit(X_train, y1_train)

best_params_stage1 = random_search_stage1.best_params_
print("Best hyperparameters for Stage 1 model:")
print(best_params_stage1)

stage1_count_model_tuned = xgb.XGBRegressor(objective='count:poisson', random_state=42, **best_params_stage1)
stage1_count_model_tuned.fit(X_train, y1_train)

print("\nTuned Stage 1 count model trained successfully.")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters for Stage 1 model:
{'alpha': np.float64(0.2721322493846353), 'colsample_bytree': np.float64(0.8590760482165449), 'lambda': np.float64(1.0010407539906316), 'learning_rate': np.float64(0.11577065690025069), 'max_depth': 5, 'n_estimators': 458, 'subsample': np.float64(0.6658623412571767)}

Tuned Stage 1 count model trained successfully.


In [ ]:
# Define parameter distributions for Stage 2 model (XGBoost Regressor)
param_dist_stage2 = {
    'n_estimators': randint(100, 500),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'lambda': uniform(1, 2),
    'alpha': uniform(0, 1)
}

random_search_stage2 = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    param_distributions=param_dist_stage2, n_iter=50,
    scoring='neg_mean_squared_error', cv=5, verbose=1, random_state=42,
    n_jobs=-1
)

random_search_stage2.fit(X_train_stage2, y2_train)

best_params_stage2 = random_search_stage2.best_params_
print("Best hyperparameters for Stage 2 model:")
print(best_params_stage2)

stage2_model_tuned = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **best_params_stage2)
stage2_model_tuned.fit(X_train_stage2, y2_train)

print("\nTuned Stage 2 model trained successfully.")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters for Stage 2 model:
{'alpha': np.float64(0.71134195274865), 'colsample_bytree': np.float64(0.9160702162124823), 'lambda': np.float64(2.2119199495620228), 'learning_rate': np.float64(0.2878902635540047), 'max_depth': 4, 'n_estimators': 140, 'subsample': np.float64(0.9659838702175123)}

Tuned Stage 2 model trained successfully.


## Summarizing results


In [ ]:
print("--- Model Performance Summary ---")

# Stage 1 Performance (potholes_per_km)
print("\nStage 1 Model (potholes_per_km) Performance:")
print(f"Mean Squared Error (MSE): {mse_potholes_per_km}")
print(f"R-squared (R2): {r2_potholes_per_km}")

# Stage 2 Performance (observed_travel_time_s)
print("\nStage 2 Model (observed_travel_time_s) Performance (on original scale):")
print(f"Mean Absolute Error (MAE): {mae_stage2}")
print(f"Mean Squared Error (MSE): {mse_stage2_original_scale}")
print(f"Root Mean Squared Error (RMSE): {rmse_stage2_original_scale}")
print(f"R-squared (R2): {r2_stage2_original_scale}")

print("\n--- Hyperparameter Tuning Insights ---")

# Best hyperparameters for Stage 1
print("\nBest hyperparameters for Stage 1 model:")
print(best_params_stage1)

# Best hyperparameters for Stage 2
print("\nBest hyperparameters for Stage 2 model:")
print(best_params_stage2)


--- Model Performance Summary ---

Stage 1 Model (potholes_per_km) Performance:
Mean Squared Error (MSE): 5.8095527013275285
R-squared (R2): 0.8061777749685815

Stage 2 Model (observed_travel_time_s) Performance (on original scale):
Mean Absolute Error (MAE): 0.5744733233833312
Mean Squared Error (MSE): 0.722791434103003
Root Mean Squared Error (RMSE): 0.8501714145412106
R-squared (R2): 0.9966511755409535

--- Hyperparameter Tuning Insights ---

Best hyperparameters for Stage 1 model:
{'alpha': np.float64(0.2721322493846353), 'colsample_bytree': np.float64(0.8590760482165449), 'lambda': np.float64(1.0010407539906316), 'learning_rate': np.float64(0.11577065690025069), 'max_depth': 5, 'n_estimators': 458, 'subsample': np.float64(0.6658623412571767)}

Best hyperparameters for Stage 2 model:
{'alpha': np.float64(0.71134195274865), 'colsample_bytree': np.float64(0.9160702162124823), 'lambda': np.float64(2.2119199495620228), 'learning_rate': np.float64(0.2878902635540047), 'max_depth': 4, 'n